In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
from gensim.test.utils import datapath
from gensim.models import word2vec, doc2vec

## Word2Vec

In [2]:
train_set = pd.read_csv('./train_set.csv', sep='\t')
X_train, X_test, y_train, y_test = train_test_split(
    train_set['text'], 
    train_set['label'], 
    random_state=42
)
# train_set['text'].to_csv('vocab.csv', index=None, header=None, sep='\t')

In [3]:
# 1w train_set
w2v_vocab = word2vec.LineSentence(
    datapath(r'D:\Python\Python Codes\★ 比赛代码\★ 学习赛\新闻分类\vocab.csv'),
    max_sentence_length=5000,
    limit=10000 # 1w
)

In [4]:
# 全量train_set，覆写word2vec.LineSentence
import itertools
class sentences(word2vec.LineSentence):
    def __init__(self, train_set, max_sentence_length=3000, limit=None):
        self.train_set = train_set
        self.max_sentence_length = max_sentence_length
        self.limit = limit
        
    def __iter__(self):
        for doc in tqdm(self.train_set):
            doc = doc.split()
            yield doc[:self.max_sentence_length]

In [5]:
w2v = word2vec.Word2Vec(
    sentences(train_set['text']),
#     w2v_vocab, # 仅1w
    vector_size=256, # 词向量的维度
    workers=6 # 调大速度会快一点，但太大会爆内存
)

  0%|          | 0/200000 [00:00<?, ?it/s]

  0%|          | 0/200000 [00:00<?, ?it/s]

  0%|          | 0/200000 [00:00<?, ?it/s]

  0%|          | 0/200000 [00:00<?, ?it/s]

  0%|          | 0/200000 [00:00<?, ?it/s]

  0%|          | 0/200000 [00:00<?, ?it/s]

In [7]:
w2v.save('./w2c_256')

Word Embedding

In [7]:
# 用 word vec 的均值作为 doc vec
def get_doc_vec(sentence, model):
    scores = np.array([model.wv[word] for word in sentence.split() if word in model.wv])
    return np.mean(scores, axis=0)
import numpy as np
X_word2vec_train = np.array([get_doc_vec(sentence, w2v) for sentence in tqdm(X_train)])
X_word2vec_test =  np.array([get_doc_vec(sentence, w2v) for sentence in tqdm(X_test)])

  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [8]:
# 保存结果
pd.concat([pd.DataFrame(X_word2vec_train),
           y_train.reset_index(drop=True)],
          axis=1).to_csv('train_emb256.csv', index=None, header=None, sep='\t')

pd.concat([pd.DataFrame(X_word2vec_test),
           y_test.reset_index(drop=True)],
          axis=1).to_csv('test_emb256.csv', index=None, header=None, sep='\t')

train and test

## Doc2Vec